# Data cleaning

先通过stata生成变量amount，num_goods, price_goods,读入数据，检查基本信息。
中国政府采购、公共采购主要分为货物、服务、工程三大类，有不同的公开招标金额标准。各省不同年份的标准不同，其中货物和服务的标准相同，工程按内容不同有不同标准。鉴于根据公开的采购信息，难以逐一判断工程类合同的具体内容，选择通过关键词判断合同是否为工程相关，直接排除工程部分，保留货物和服务采购数据。
下面的程序主要进行以下数据清洗：
1. 提取"项目名称","采购人甲方","采购人地址"中的省市县信息，结合2020年中国省市县三级行政区划表，匹配采购数据的地区(省级)
2. 根据"主要标的名称"和"项目名称"中词汇判断采购类型。
   出现的词频，选出top 100关键词，对关键词进行手动分类，排除其中容易出现歧义的部分，比如油，可能是“燃油采购”（货物），也可能是“加油服务”（服务）。对包含关键词的采购项目进行分类，在通过机器学习进行分类。

中国政府采购分为分散采购和集中采购两种，在大部分省份，当采购金额低于50万元时由地方分散采购，所以在我们的数据中，保留了50万元以上的采购项目。各省集中采购公开招标的门槛最大为400万元，以100万元为基础的带宽，保留数据到500万元。

In [30]:
import pandas as pd

pd.set_option("display.max_columns", None)   
pd.set_option("display.max_rows", 100)
pd.set_option("display.width", None)     

In [ ]:

csv_file = "/Users/yxy/UChi/Summer2025/Procurement/dta/china_procurement_clean1.csv"

df = pd.read_csv(csv_file, low_memory=False)

df_filtered = df[(df['amount'] >= 50) & (df['amount'] <= 500) & (df['年份'] >= 2020)]

In [ ]:
df_filtered['年份'].value_counts()

In [ ]:
df_filtered['amount'].describe()

## 确定省份

In [21]:
import geopandas as gpd

shp_path = "/Users/yxy/UChi/Summer2025/Procurement/raw/Countylevel_Admin_2020/China2020County.shp"
gdf = gpd.read_file(shp_path)

gdf = gdf[['省级', '地级', '县级']]

In [20]:
import re
def extract_prov_city_county(text: str):
    if not isinstance(text, str):  
        return None, None, None
    text = re.sub(r"[-_\s·、，,\.]()（）*。", "", text) 
    # text = re.sub(r"[^\u4e00-\u9fa5]", "", text)

    prov_pattern = r"(.*?(省|自治区|市))"    
    city_pattern = r"(.*?(市|地区|盟|州))"  
    county_pattern = r"(.*?(县|区|旗))"     
    
    prov, city, county = None, None, None
    municipalities = ["北京市","天津市","上海市","重庆市"]

    prov_match = re.match(prov_pattern, text)
    if prov_match:
        if '市' in prov_match.group(1) and prov_match.group(1) not in municipalities:
            prov = None
        else: 
            prov = prov_match.group(1)
            text = text[len(prov):]
    city_match = re.match(city_pattern, text)
    if city_match:
        city_candidate = city_match.group(1)
        if city_candidate.endswith("州") and text.startswith("市", len(city_candidate)):
            city = city_candidate + "市"
            text = text[len(city_candidate) + 1:] 
        else:
            city = city_candidate
            text = text[len(city):]
        
    county_match = re.match(county_pattern, text)
    if county_match:
        county = county_match.group(1)

    return prov, city, county


In [22]:

def fill_location(row):
    prov, city, county = None, None, None

    p1, c1, ct1 = extract_prov_city_county(row["采购人地址"])
    prov, city, county = p1, c1, ct1

    if prov is None or city is None or county is None:
        p2, c2, ct2 = extract_prov_city_county(row["采购人甲方"])
        if prov is None: prov = p2
        if city is None: city = c2
        if county is None: county = ct2

    if prov is None or city is None or county is None:
        p3, c3, ct3 = extract_prov_city_county(row["项目名称"])
        if prov is None: prov = p3
        if city is None: city = c3
        if county is None: county = ct3

    return pd.Series([prov, city, county])


In [23]:
def match_region(row, gdf):
    if row["prov"]:
        match = gdf[gdf["省级"] == row["prov"]]
        if not match.empty:
            return match["省级"].iloc[0]
    if row["city"]:
        match = gdf[gdf["地级"].str.contains(str(row["city"]).replace("市",""), na=False, regex=False)]
        if not match.empty:
            return match["省级"].iloc[0]
        match = gdf[gdf["县级"].str.contains(str(row["city"]).replace("市",""), na=False, regex=False)]
        if not match.empty:
            return match["省级"].iloc[0]
    if row["county"]:
        match = gdf[gdf["县级"] == row["county"]]
        if not match.empty:
            return match["省级"].iloc[0]

    return None


In [ ]:
df_filtered[["prov", "city", "county"]] = df_filtered.apply(fill_location, axis=1)
df_filtered["region"] = df_filtered.apply(lambda x: match_region(x, gdf), axis=1)
df_filtered.to_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/china_procurement_region.csv", index=False, encoding="utf-8-sig")

In [25]:
df_filtered = pd.read_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/china_procurement_region.csv", low_memory=False)

In [24]:
import jieba

def match_region_by_jieba(row, gdf):
    fields = ["采购人地址", "采购人甲方", "项目名称"]

    for field in fields:
        text = row.get(field, "")
        if not isinstance(text, str) or text.strip() == "":
            continue

        words = jieba.lcut(text)
        if not words:
            continue
        first_word = words[0]

        match = gdf[gdf["省级"].str.contains(first_word, na=False, regex=False)]
        if not match.empty:
            return match["省级"].iloc[0]

    return None

In [26]:
mask = df_filtered["region"].isna()
df_filtered.loc[mask, "region"] = df_filtered[mask].apply(
    lambda x: match_region_by_jieba(x, gdf), axis=1
)
df_filtered = df_filtered[
    ~df_filtered["region"].isin(["澳门特别行政区", "台湾省"])
]
df_filtered.to_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/china_procurement_region2.csv", index=False, encoding="utf-8-sig")

## 确定类型
### top 200 items
给出现频率前200的标的物名称手动标注类别，使用了chatgpt+人工检查，对一部分“无”，“详情见合同”，标注了类别 “无分类”

In [27]:
out_path_kw = "/Users/yxy/UChi/Summer2025/Procurement/dta/keywords.csv"

In [46]:
df_filtered = pd.read_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/china_procurement_region.csv", low_memory=False)

In [47]:
df_filtered = df_filtered.dropna(subset=["region"])

In [49]:
df_filtered['年份'].value_counts(dropna=False)

年份
2023    191582
2022    158495
2021    105770
2020     47632
2024     24300
Name: count, dtype: int64

In [50]:
import pandas as pd
out_path_kw = "/Users/yxy/UChi/Summer2025/Procurement/dta/keywords.csv"
top100_items = df_filtered["主要标的名称"].value_counts().head(200).reset_index()
top100_items.columns = ["keyword", "count"]

top100_items["category"] = ""

top100_items.to_csv(out_path_kw, index=False, encoding="utf-8-sig")

print("Top 200 items exported to keywords.csv for manual categorization.")

Top 200 items exported to keywords.csv for manual categorization.


In [52]:
classified = pd.read_csv(out_path_kw)
df_filtered = df_filtered.merge(classified[["keyword", "category"]], 
              left_on="主要标的名称", 
              right_on="keyword", 
              how="left")
df_filtered.rename(columns={"category": "cat"}, inplace=True)
df_filtered.drop(columns=["keyword"], inplace=True)

In [57]:
df_filtered.loc[df_filtered["cat"].isna(), "主要标的名称"].value_counts(dropna=False)


主要标的名称
物业管理服务,采购数量1;                                187
物业服务,采购数量1;                                  107
详见合同文件                                        81
食堂食材                                          80
物业管理服务,采购数量1.0000;                            79
                                            ... 
侵蚀沟水毁工程苗木补植                                    1
CT室、介入手术室、MR室GE设备维保服务                          1
卢龙县2023年度石门镇等9个乡镇土地整治（占补平衡）项目勘测、设计及预算编制B包      1
卢龙县2023年度石门镇等9个乡镇土地整治（占补平衡）项目勘测、设计及预算编制A包      1
NaI探测器                                         1
Name: count, Length: 358479, dtype: int64

In [60]:
import pandas as pd

# 读入两个csv
df1 = pd.read_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/keywords.csv")   
df2 = pd.read_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/cat_list.csv")  

df1 = df1[["keyword", "category"]]
df2 = df2[["keyword", "category"]]

df_all = pd.concat([df1, df2], ignore_index=True).drop_duplicates(subset=["keyword"])

df_all.to_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/keywords_with_cat.csv", index=False)


In [86]:
import pandas as pd
import jieba
import re
import numpy as np


kw_df = pd.read_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/keywords_with_cat.csv")

def clean_text(text):
    if not isinstance(text, str):
        return ""
    return re.sub(r"[^\u4e00-\u9fa50-9]", "", text)

def match_cat_from_text(text, kw_df):
    kw_df = kw_df.rename(columns={"category": "cat"})
    text = clean_text(text)
    if not text:
        return None
    words = jieba.lcut(text)
    for w in words:
        if w == "项目":
            continue
        match = kw_df[kw_df["keyword"].str.contains(w, na=False)]
        if not match.empty:
            cat = match["cat"].iloc[0]
            if cat in ["工程", "货物", "服务"]:
                return cat
    return None

def assign_cat(row, kw_df):
    for col in ["主要标的物名称", "项目名称"]:
        text = row.get(col, "")
        cat = match_cat_from_text(text, kw_df)
        if cat is not None:
            return cat
    return None



In [65]:
mask = df_filtered["cat"].isna() | (df_filtered["cat"] == "无分类")

df_filtered.loc[mask, "cat"] = df_filtered.loc[mask].apply(
    lambda x: assign_cat(x, kw_df), axis=1
)


In [88]:
df_filtered['cat'].value_counts(dropna=False)

cat
服务      233571
货物      154493
None     76262
工程       75534
Name: count, dtype: int64

In [91]:
text = clean_text('气实验室设备+噪声设备+土壤设备')
words = jieba.lcut(text)
words

['气', '实验室', '设备', '噪声', '设备', '土壤', '设备']

In [87]:
mask = df_filtered["cat"].isna()


df_filtered.loc[mask, "cat"] = df_filtered.loc[mask].apply(
    lambda x: assign_cat(x, kw_df), axis=1
)

In [89]:
df_unmatched = df_filtered[df_filtered['cat'].isna()]
df_unmatched.sample(10)

,合同编号,项目编号,项目名称,采购人甲方,采购人地址,采购人联系方式,供应商乙方,供应商地址,供应商联系方式,主要标的名称,规格型号或服务要求,主要标的数量,主要标的单价,合同金额万元,履约期限地点等简要信息,采购方式,合同签订日期,合同公告日期,年份,amount,num_goods,price_goods,prov,city,county,region,cat
80562,HBZJ-BD2021E158,HB2021113610140008,保定市第一中心医院新建总院疫情防控临建用房项目,保定市第一中心医院,保定市第一中心医院,0312-5976526,河北中晔建筑工程有限公司,保定市竞秀区韩村北路街道办事处七一中路1616号九州商务中心3A07室,0312-3230986,保定市第一中心医院新建总院疫情防控临建用房项目,图纸及工程量清单中全部内容,1项,2114410,211.4410,履约期限自签订合同之日起90日 地点保定市第一中心医院,NaN,2022-01-19,2022-01-20,2022,211.44099,NaN,2114410.0,NaN,保定市,NaN,河北省,None
185249,STHLH（G）-2022-079,STHLH（G）-2022-079,武功县渭河北岸废弃矿山生态修复项目,武功县自然资源局,武功县普集街道办长青北路,18690086589,陕西联信建设工程有限公司,陕西省咸阳市秦都区秦皇路外滩一号3号楼1层01号,13109633171,武功县渭河北岸废弃矿山生态修复,详见合同,1.0000项,3969336.0000元,396.9336,武功,竞争性磋商,2022-11-16,2023-05-16,2023,396.93359,NaN,NaN,NaN,NaN,武功县,陕西省,None
535618,HT_SZCG2022000138-A,SZCG2022000138,深圳市环境监测点位定位与生态标识项目,深圳市环境监测中心站,深圳市福田区梅坳七路8号,15818673962,深圳市城市规划设计研究院有限公司,广东省深圳市南山区留仙大道创智云城A4栋10楼,18825049893,深圳市城市规划设计研究院有限公司,项,1,1061000,106.1000,2022年4月21日至2023年3月31日,公开招标,2022-04-21,2022-04-22,2022,106.10000,1.0,1061000.0,NaN,深圳市,福田区,广东省,None
151897,ZY140G2102H20021,ZY140G2102H20021,广州市何贤纪念医院住院部发电机采购安装项目,广州市番禺区何贤纪念医院（广州市番禺区妇幼保健院）,广东省广州市番禺区清河东路2号,18026281988,广州筑安机电设备安装有限公司,广州市番禺区桥南街南堤东路784号、786号、788号首层,020-84649188,广州市何贤纪念医院住院部发电机采购安装项目,详见合同,1.00项,1763000.00元,176.3000,广州市番禺区市桥,公开招标,2022-12-19,2022-12-26,2022,176.30000,NaN,NaN,广东省,广州市,番禺区,广东省,None
440272,N5109042022000074-1,N5109042022000074,遂宁市安居区2022年中央财政农业发展资金小麦绿色高质高效行动项目(二次),遂宁市安居区农业农村局,遂宁市安居区国贸西路2号,13408254939,成都垦土农业科技有限公司,四川省成都市锦江区华兴正街5号2栋17层3号,028-85543983,复混肥料,详见合同,450.0900吨,3266.013400元,147.0000,不详,竞争性磋商,2023-02-13,2023-02-15,2023,147.00000,NaN,NaN,NaN,遂宁市,安居区,四川省,None
447090,LXZC11920230032-1,LXZC11920230032,永靖旱作农业综合丰产技术推广项目,永靖县农业农村局,无,无,兰州金土地塑料制品有限公司,无,无,加厚高强度白地膜,宽度为1200mm，厚度为0.015mm。,289吨,10020元,289.5780,履约开始日期2023-03-08;履约截止日期2023-03-24;计划验收日期;履约地点永...,公开招标,2023-03-08,2023-03-17,2023,289.57800,NaN,NaN,NaN,NaN,永靖县,甘肃省,None
351884,海采（2021）0748号/TXJ-010-2021199,海采（2021）0748号/TXJ-010-2021199,2021年马连洼街道背街小巷降尘作业试点项目,北京市海淀区人民政府马连洼街道办事处,北京市海淀区马连洼北路8号院,62829865,北京京环高科城市环境管理有限公司,北京市海淀区四季青路8号2层221-2,62944708,2021年马连洼街道背街小巷降尘作业试点项目,无,1,无,155.4950,2021年马连洼街道背街小巷；自合同签订之日起6个月,竞争性磋商,2021-09-03,2022-01-13,2022,155.49500,1.0,NaN,北京市,NaN,海淀区,北京市,None
247034,ZCSP-渭南市-2023-00924,ZCSP-渭南市-2023-00924,2021年省级高技能人才项目,渭南技师学院,渭南市开发区东新街15号,0913-2116190,西安诚与慧实业有限公司,陕西省西安市未央区未央路 80 号盛龙广场 A 区 1 号楼 2 单元 1708 室,13700226379,2021年省级高技能人才,详见合同,1.0000批,775000.0000元,77.5000,渭南技师学院,公开招标,2023-12-15,2023-12-15,2023,77.50000,NaN,NaN,2021年省,渭南市,开发区,陕西省,None
454311,N5106012023000018-4,N5106012023000018,德阳市县级生态环境监测机构标准化建设项目,德阳市生态环境局,德阳,0838-2314651,成都汇泓科技有限公司,成都市金牛区树蓓街76号1楼,028-87535698,气实验室设备+噪声设备+土壤设备,详见合同,1.0000批,1283200.000000元,128.3200,不详,公开招标,2023-04-20,2023-04-20,2023,128.32001,NaN,NaN,NaN,德阳市,县,四川省,None
511073,HBBJ-ZJK-(2023)011,HB2023103240010051,康保县2023年省级财政森林抚育项目（森林质量精准提升）,康保县自然资源和规划局本级,康保县自然资源和规划局本级,0313-7638952,安徽四季春建设有限公司,安徽省马鞍山市含山县一统碑行政村,0551-62692273,康保县2023年省级财政森林抚育项目（森林质量精准提升）,疏伐2434.53亩，定株2078.4亩，补植5487.07亩。（具体内容详见工程量清单）,1,1850500,185.0500,2024年6月30日前完工；实施地点为招标人指定地点。,NaN,2023-11-13,2023-11-14,2023,185.05000,1.0,1850500.0,康保县2023年省,NaN,康保县,河北省,None


In [92]:
df_filtered.to_csv("/Users/yxy/UChi/Summer2025/Procurement/dta/china_procurement_primary.csv", index=False, encoding="utf-8-sig")